In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'D2Coding'
plt.rcParams['font.size'] = 15
import seaborn as sns
from plotnine import *
import missingno as msno
import folium
import requests

In [2]:
width = pd.read_csv('./data/동북부면적.csv')
width = width.drop(columns=['Unnamed: 0'], axis=1)
width

,시군구,법정동,면적
0,동대문구,신설동,0.22
1,동대문구,용두동,0.76
2,동대문구,제기동,0.72
3,동대문구,전농동,0.94
4,동대문구,답십리동,1.02
5,동대문구,장안동,1.47
6,동대문구,청량리동,0.62
7,동대문구,회기동,0.28
8,동대문구,휘경동,0.86
9,동대문구,이문동,0.91


In [3]:
# 네이버 개발자 포털에서 발급한 Client ID와 Client Secret
client_id = '5j924usqjd'
client_secret = 'W4wjDI2jRzuvP1N0zo9UiG1zTRoDP30idjpESbp4'

# '위도'와 '경도' 칼럼을 추가합니다.
width['위도'] = None
width['경도'] = None

# 네이버 지오코딩 API를 호출하여 주소별 위도와 경도를 얻습니다.
for index, row in width.iterrows():
    address = row['법정동']
    url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address}'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('addresses'):
            latitude = data['addresses'][0]['y']
            longitude = data['addresses'][0]['x']
            width.at[index, '위도'] = latitude
            width.at[index, '경도'] = longitude

# 결과를 출력합니다.
width

,시군구,법정동,면적,위도,경도
0,동대문구,신설동,0.22,37.5749343,127.0255251
1,동대문구,용두동,0.76,37.5758884,127.0372011
2,동대문구,제기동,0.72,37.5831612,127.0378551
3,동대문구,전농동,0.94,37.5779984,127.0477575
4,동대문구,답십리동,1.02,37.5717963,127.0512678
5,동대문구,장안동,1.47,37.567842,127.066375
6,동대문구,청량리동,0.62,37.5831766,127.0484354
7,동대문구,회기동,0.28,37.5908083,127.0553132
8,동대문구,휘경동,0.86,37.592932,127.065725
9,동대문구,이문동,0.91,37.597849,127.06545


In [4]:
academy = pd.read_csv('./data/동북부학원수.csv')
academy = academy.drop(columns=['Unnamed: 0'], axis=1)
academy

,시군구,법정동,면적(㎢)당 학원 수
0,강북구,미아동,24.907063
1,강북구,번동,10.891089
2,강북구,수유동,15.040650
3,강북구,우이동,3.846154
4,노원구,공릉동,15.337423
5,노원구,상계동,26.041667
6,노원구,월계동,13.235294
7,노원구,중계동,113.450292
8,노원구,하계동,20.731707
9,도봉구,도봉동,4.504505


In [11]:
house = pd.read_csv('./data/동북부집값.csv')
house = house.drop(columns=['Unnamed: 0'], axis=1)
house

,시군구,법정동,면적(㎡)당금액
0,강북구,미아동,990.0
1,강북구,번동,922.0
2,강북구,수유동,965.0
3,강북구,우이동,755.0
4,노원구,공릉동,1016.0
5,노원구,상계동,1115.0
6,노원구,월계동,1213.0
7,노원구,중계동,1169.0
8,노원구,하계동,1222.0
9,도봉구,도봉동,784.0


In [12]:
house

,시군구,법정동,면적(㎡)당금액
0,강북구,미아동,990.0
1,강북구,번동,922.0
2,강북구,수유동,965.0
3,강북구,우이동,755.0
4,노원구,공릉동,1016.0
5,노원구,상계동,1115.0
6,노원구,월계동,1213.0
7,노원구,중계동,1169.0
8,노원구,하계동,1222.0
9,도봉구,도봉동,784.0


In [13]:
# 카카오맵 API Key
kakao_api_key = '146c824df784569973cb17777ddd3301'

# '위도'와 '경도' 칼럼을 추가합니다.
house['위도'] = None
house['경도'] = None

# 카카오맵 API를 호출하여 주소별 위도와 경도를 얻습니다.
for index, row in house.iterrows():
    address = row['법정동']
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
    headers = {
        'Authorization': f'KakaoAK {kakao_api_key}'
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('documents'):
            latitude = data['documents'][0]['y']
            longitude = data['documents'][0]['x']
            house.at[index, '위도'] = latitude
            house.at[index, '경도'] = longitude

# 결과를 출력합니다.
house


,시군구,법정동,면적(㎡)당금액,위도,경도
0,강북구,미아동,990.0,37.6270831294022,127.027060776679
1,강북구,번동,922.0,37.6241057350225,127.04133717608
2,강북구,수유동,965.0,37.6424621841483,127.022094520762
3,강북구,우이동,755.0,37.6630390800114,127.001890452891
4,노원구,공릉동,1016.0,37.6248431089169,127.073821318894
5,노원구,상계동,1115.0,37.66125057554,127.060252770712
6,노원구,월계동,1213.0,37.6199191869585,127.062987742987
7,노원구,중계동,1169.0,37.6525550467648,127.077488598779
8,노원구,하계동,1222.0,37.6404885286244,127.072685646642
9,도봉구,도봉동,784.0,37.6873960128039,127.021495626589


위도 경도를 넣어서 행정동 출력

In [20]:
house_test = house[:2]
house_test

,시군구,법정동,면적(㎡)당금액,위도,경도,행정동,행정코드,법정코드
0,강북구,미아동,990.0,37.6270831294022,127.027060776679,미아동,None,None
1,강북구,번동,922.0,37.6241057350225,127.04133717608,번동,None,None


In [21]:
# 카카오맵 API Key
kakao_api_key = '146c824df784569973cb17777ddd3301'

house_test['행정동'] = None

# 카카오맵 API를 호출하여 주소별 위도, 경도 및 행정동명을 얻습니다.
for index, row in house_test.iterrows():
    latitude = row['위도']
    longitude = row['경도']
    
    url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'
    headers = {
        'Authorization': f'KakaoAK {kakao_api_key}'
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        print(data)
        if data.get('documents'):
            administrative_district1 = data['documents'][0]['region_3depth_name']
#             administrative_district2 = data['documents'][1]['h_code']
#             administrative_district3 = data['documents'][2]['b_code']
            
            house_test.at[index, '행정동'] = administrative_district1
#             house_test.at[index, '행정코드'] = administrative_district2
#             house_test.at[index, '법정코드'] = administrative_district3

# 결과를 출력합니다.
house_test


{'meta': {'total_count': 2}, 'documents': [{'region_type': 'B', 'code': '1130510100', 'address_name': '서울특별시 강북구 미아동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '강북구', 'region_3depth_name': '미아동', 'region_4depth_name': '', 'x': 127.02706077667905, 'y': 37.62708312940752}, {'region_type': 'H', 'code': '1130553500', 'address_name': '서울특별시 강북구 미아동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '강북구', 'region_3depth_name': '미아동', 'region_4depth_name': '', 'x': 127.02706077667905, 'y': 37.62708312940752}]}
{'meta': {'total_count': 2}, 'documents': [{'region_type': 'B', 'code': '1130510200', 'address_name': '서울특별시 강북구 번동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '강북구', 'region_3depth_name': '번동', 'region_4depth_name': '', 'x': 127.02887295066942, 'y': 37.63792696525825}, {'region_type': 'H', 'code': '1130560800', 'address_name': '서울특별시 강북구 번3동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '강북구', 'region_3depth_name': '번3동', 'region_4depth_name': '', '

,시군구,법정동,면적(㎡)당금액,위도,경도,행정동,행정코드,법정코드
0,강북구,미아동,990.0,37.6270831294022,127.027060776679,미아동,None,None
1,강북구,번동,922.0,37.6241057350225,127.04133717608,번동,None,None


In [23]:
# 카카오맵 API Key
kakao_api_key = '146c824df784569973cb17777ddd3301'
# '위도'와 '경도' 칼럼을 추가합니다.
house['행정동'] = None

# 카카오맵 API를 호출하여 주소별 위도, 경도 및 행정동명을 얻습니다.
for index, row in house.iterrows():
    latitude = row['위도']
    longitude = row['경도']
    
    url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'
    headers = {
        'Authorization': f'KakaoAK {kakao_api_key}'
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('documents'):
            administrative_district = data['documents'][1]['region_3depth_name']
            
            house.at[index, '행정동'] = administrative_district

# 결과를 출력합니다.
house


{'meta': {'total_count': 2}, 'documents': [{'region_type': 'B', 'code': '1130510100', 'address_name': '서울특별시 강북구 미아동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '강북구', 'region_3depth_name': '미아동', 'region_4depth_name': '', 'x': 127.02706077667905, 'y': 37.62708312940752}, {'region_type': 'H', 'code': '1130553500', 'address_name': '서울특별시 강북구 미아동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '강북구', 'region_3depth_name': '미아동', 'region_4depth_name': '', 'x': 127.02706077667905, 'y': 37.62708312940752}]}
{'meta': {'total_count': 2}, 'documents': [{'region_type': 'B', 'code': '1130510200', 'address_name': '서울특별시 강북구 번동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '강북구', 'region_3depth_name': '번동', 'region_4depth_name': '', 'x': 127.02887295066942, 'y': 37.63792696525825}, {'region_type': 'H', 'code': '1130560800', 'address_name': '서울특별시 강북구 번3동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '강북구', 'region_3depth_name': '번3동', 'region_4depth_name': '', '

{'meta': {'total_count': 2}, 'documents': [{'region_type': 'B', 'code': '1123010600', 'address_name': '서울특별시 동대문구 장안동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '동대문구', 'region_3depth_name': '장안동', 'region_4depth_name': '', 'x': 127.06667914396226, 'y': 37.56777497225432}, {'region_type': 'H', 'code': '1123066000', 'address_name': '서울특별시 동대문구 장안2동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '동대문구', 'region_3depth_name': '장안2동', 'region_4depth_name': '', 'x': 127.0706147287471, 'y': 37.57845848177183}]}
{'meta': {'total_count': 2}, 'documents': [{'region_type': 'B', 'code': '1123010400', 'address_name': '서울특별시 동대문구 전농동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '동대문구', 'region_3depth_name': '전농동', 'region_4depth_name': '', 'x': 127.04775995453092, 'y': 37.57797259208786}, {'region_type': 'H', 'code': '1123056000', 'address_name': '서울특별시 동대문구 전농1동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '동대문구', 'region_3depth_name': '전농1동', 'region_4depth

{'meta': {'total_count': 2}, 'documents': [{'region_type': 'B', 'code': '1126010400', 'address_name': '서울특별시 중랑구 묵동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '중랑구', 'region_3depth_name': '묵동', 'region_4depth_name': '', 'x': 127.0784707393777, 'y': 37.61235962606898}, {'region_type': 'H', 'code': '1126062000', 'address_name': '서울특별시 중랑구 묵1동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '중랑구', 'region_3depth_name': '묵1동', 'region_4depth_name': '', 'x': 127.07848438732296, 'y': 37.612413676315946}]}
{'meta': {'total_count': 2}, 'documents': [{'region_type': 'B', 'code': '1126010200', 'address_name': '서울특별시 중랑구 상봉동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '중랑구', 'region_3depth_name': '상봉동', 'region_4depth_name': '', 'x': 127.08724964761423, 'y': 37.59989457757289}, {'region_type': 'H', 'code': '1126059000', 'address_name': '서울특별시 중랑구 상봉2동', 'region_1depth_name': '서울특별시', 'region_2depth_name': '중랑구', 'region_3depth_name': '상봉2동', 'region_4depth_name': '',

,시군구,법정동,면적(㎡)당금액,위도,경도,행정동,행정코드,법정코드
0,강북구,미아동,990.0,37.6270831294022,127.027060776679,미아동,None,None
1,강북구,번동,922.0,37.6241057350225,127.04133717608,번3동,None,None
2,강북구,수유동,965.0,37.6424621841483,127.022094520762,수유2동,None,None
3,강북구,우이동,755.0,37.6630390800114,127.001890452891,우이동,None,None
4,노원구,공릉동,1016.0,37.6248431089169,127.073821318894,공릉1동,None,None
5,노원구,상계동,1115.0,37.66125057554,127.060252770712,상계10동,None,None
6,노원구,월계동,1213.0,37.6199191869585,127.062987742987,월계1동,None,None
7,노원구,중계동,1169.0,37.6525550467648,127.077488598779,중계1동,None,None
8,노원구,하계동,1222.0,37.6404885286244,127.072685646642,하계1동,None,None
9,도봉구,도봉동,784.0,37.6873960128039,127.021495626589,도봉1동,None,None


In [ ]:
house.columns = ['시군구', '법정동', 'price', 'latitude', 'longitude']
house

In [ ]:
condition = house['법정동'] == '시가지동'
house.loc[condition, 'latitude'] = 37.592777
house.loc[condition, 'longitude'] = 127.01651385
house

In [ ]:
house['latitude'] = house['latitude'].str.replace(',', '').astype(float)
house['longitude'] = house['longitude'].str.replace(',', '').astype(float)
house

In [ ]:
house.to_csv('./data/동북부집값위경도.csv')
house

In [7]:
df = pd.read_csv('./data/동북부집값위경도.csv')
df

,Unnamed: 0,시군구,법정동,price,latitude,longitude
0,0,강북구,미아동,990.0,37.627074,127.026979
1,1,강북구,번동,922.0,37.637891,127.028847
2,2,강북구,수유동,965.0,37.630086,127.017601
3,3,강북구,우이동,755.0,37.663300,127.002000
4,4,노원구,공릉동,1016.0,37.624874,127.073838
5,5,노원구,상계동,1115.0,37.679901,127.054978
6,6,노원구,월계동,1213.0,37.619920,127.063017
7,7,노원구,중계동,1169.0,37.652599,127.077528
8,8,노원구,하계동,1222.0,37.640537,127.072585
9,9,도봉구,도봉동,784.0,37.678691,127.043437


In [8]:
condition = df['법정동'] == '시가지동'
df.loc[condition, 'latitude'] = 37.592777
df.loc[condition, 'longitude'] = 127.01651385
df

,Unnamed: 0,시군구,법정동,price,latitude,longitude
0,0,강북구,미아동,990.0,37.627074,127.026979
1,1,강북구,번동,922.0,37.637891,127.028847
2,2,강북구,수유동,965.0,37.630086,127.017601
3,3,강북구,우이동,755.0,37.663300,127.002000
4,4,노원구,공릉동,1016.0,37.624874,127.073838
5,5,노원구,상계동,1115.0,37.679901,127.054978
6,6,노원구,월계동,1213.0,37.619920,127.063017
7,7,노원구,중계동,1169.0,37.652599,127.077528
8,8,노원구,하계동,1222.0,37.640537,127.072585
9,9,도봉구,도봉동,784.0,37.678691,127.043437


In [10]:
data = df
house_map = folium.Map(location=[37.6199197, 127.0630165], zoom_start=12)

for index, data in data.iterrows():
    popup = folium.Popup('{}: {:,.0f}'.format(data.법정동, data.price), max_width=300)
    folium.CircleMarker(location=[data.latitude, data.longitude], popup=popup, radius=data.price / 100, 
                        color='red',fill_color='blue').add_to(house_map)

house_map

In [ ]:
school = pd.read_csv('./data/동북부학교수.csv')
school = school.drop(columns=['Unnamed: 0'], axis=1)
school